In [6]:
import torch
from tokenizer import TokenizerWrapper, download_and_merge_text_files, train_tokenizer,download_file_from_url
from dataset_loader import TextDataset, collate_fn
from gru_model import GRULanguageModel
from train_utils import train_model, evaluate_model

In [7]:
# main.py

import torch
from tokenizer import TokenizerWrapper, download_and_merge_text_files, train_tokenizer
from dataset_loader import TextDataset, collate_fn
from train_utils import train_model, evaluate_model

# Import all models
from gru_model import GRULanguageModel
from lstm_model import LSTMLanguageModel
from rnn_model import RNNLanguageModel
from transformer_model import TransformerLanguageModel

# --- Config ---
DATA_URL = "https://api.github.com/repos/jghawaly/CSC7809_FoundationModels/contents/Project2/data/raw"
CORPUS_FILE = "corpus.txt"
TRAIN_FILE = "train.jsonl"
TEST_FILE = "test.jsonl"
TOKENIZER_PREFIX = "bpe_tokenizer"
VOCAB_SIZE = 10000
MAX_SEQ_LEN = 128
BATCH_SIZE = 256
EPOCHS = 50
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Select Model Here ===
MODEL_TYPE = "transformer"  # Options: 'gru', 'lstm', 'rnn', 'transformer'
MODEL_SAVE_PATH = f"best_{MODEL_TYPE}_model.pt"

# --- Step 1: Download data & train tokenizer ---
download_and_merge_text_files(DATA_URL, CORPUS_FILE)
train_tokenizer(CORPUS_FILE, TOKENIZER_PREFIX, vocab_size=VOCAB_SIZE)
tokenizer = TokenizerWrapper(f"{TOKENIZER_PREFIX}.model")

# --- Step 2: Dataset ---
train_dataset = TextDataset(TRAIN_FILE, tokenizer, MAX_SEQ_LEN)
test_dataset = TextDataset(TEST_FILE, tokenizer, MAX_SEQ_LEN)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_fn(b, tokenizer.get_pad_id()))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda b: collate_fn(b, tokenizer.get_pad_id()))


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: corpus.txt
  input_format: 
  model_prefix: bpe_tokenizer
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <bos>
  user_defined_symbols: <eos>
  user_defined_symbols: <pad>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_p

In [1]:

# --- Step 3: Model Factory ---
def build_model(model_type):
    if model_type == "gru":
        return GRULanguageModel(VOCAB_SIZE, 256, 512, 2, tokenizer.get_pad_id()).to(DEVICE)
    elif model_type == "lstm":
        return LSTMLanguageModel(VOCAB_SIZE, 256, 512, 2, tokenizer.get_pad_id()).to(DEVICE)
    elif model_type == "rnn":
        return RNNLanguageModel(VOCAB_SIZE, 256, 512, 2, tokenizer.get_pad_id()).to(DEVICE)
    elif model_type == "transformer":
        return TransformerLanguageModel(
            vocab_size=VOCAB_SIZE,
            embed_dim=256,
            num_heads=4,
            num_layers=4,
            pad_token_id=tokenizer.get_pad_id()
        ).to(DEVICE)
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

model = build_model(MODEL_TYPE)

# --- Step 4: Train & Evaluate ---
train_model(model, train_loader, test_loader, tokenizer, DEVICE, MODEL_SAVE_PATH, lr=1e-3, epochs=EPOCHS)
evaluate_model(model, MODEL_SAVE_PATH, test_loader, tokenizer, DEVICE)

# --- Step 5: Sample Generation ---
custom_prompts = [
    "The spaceship landed on the surface of Mars and",
    "He walked into the room, completely unaware that",
    "Long ago in a forgotten village, a child was born who",
    "Artificial intelligence will change the world when",
    "What do you prefer — cat or dog?"
]

print(f"\n--- Generations using {MODEL_TYPE.upper()} ---")
for prompt in custom_prompts:
    output = model.generate(tokenizer, prompt, device=DEVICE, return_continuation_only=True)
    print(f" Prompt    : {prompt}")
    print(f" Generated : {output}")


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: corpus.txt
  input_format: 
  model_prefix: bpe_tokenizer
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <bos>
  user_defined_symbols: <eos>
  user_defined_symbols: <pad>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_p

Epoch 1 | Train Loss: 6.5641 | Val Loss: 5.9267
✅ Model saved to best_transformer_model.pt
📈 Saved loss curve as: TransformerLanguageModel_loss_curve_20250416_025711.png

🧪 Test Perplexity: 374.9277
🧪 Average BLEU Score: 0.0073

--- Generations using TRANSFORMER ---

📝 Prompt    : The spaceship landed on the surface of Mars and
🧠 Generated : began with a path took the tomb and the terms off. I exampleously is servants end, and they were continually and astonished who made a broken God that saw the sailorsvents of their friends of the future in the herald through anyone. A Py

📝 Prompt    : He walked into the room, completely unaware that
🧠 Generated : she expected. Oh, anxious the But when better not suppose a hundred used go with a buck breadon, and all him that the fisher is due from the Sp falseance of events and related, took him to Mrs", a time, now

📝 Prompt    : Long ago in a forgotten village, a child was born who
🧠 Generated : ushes a tri

📝 Prompt    : Artificial intelligence

In [8]:
# Define model-specific hyperparameter grids
hyperparams_grid = {
    "gru": [
        {"embed_dim": 128, "hidden_dim": 256, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 256, "hidden_dim": 512, "num_layers": 3, "lr": 5e-4}
    ],
    "lstm": [
        {"embed_dim": 128, "hidden_dim": 256, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 256, "hidden_dim": 512, "num_layers": 3, "lr": 1e-4}
    ],
    "rnn": [
        {"embed_dim": 128, "hidden_dim": 128, "num_layers": 2, "lr": 1e-3}
    ],
    "transformer": [
        {"embed_dim": 256, "num_heads": 4, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 512, "num_heads": 8, "num_layers": 4, "lr": 5e-4}
    ]
}


In [9]:
from train_utils import train_model, evaluate_model
import pandas as pd
import time

results = []

def run_experiments(model_type, ModelClass, grid, tokenizer, train_loader, test_loader, device):
    for idx, config in enumerate(grid):
        model_id = f"{model_type}_{idx}_{int(time.time())}"
        print(f"\n Training {model_id} with config: {config}")
        
        if model_type == "transformer":
            model = ModelClass(
                vocab_size=VOCAB_SIZE,
                embed_dim=config["embed_dim"],
                num_heads=config["num_heads"],
                num_layers=config["num_layers"],
                pad_token_id=tokenizer.get_pad_id()
            ).to(device)
        else:
            model = ModelClass(
                vocab_size=VOCAB_SIZE,
                embed_dim=config["embed_dim"],
                hidden_dim=config["hidden_dim"],
                num_layers=config["num_layers"],
                pad_token_id=tokenizer.get_pad_id()
            ).to(device)
        
        save_path = f"{model_id}.pt"
        train_model(model, train_loader, test_loader, tokenizer, device, save_path, lr=config["lr"], epochs=50)

        ppl, bleu = evaluate_model(model, save_path, test_loader, tokenizer, device)
        results.append({
            "model_type": model_type,
            "config": config,
            "perplexity": ppl,
            "bleu_score": bleu,
            "model_path": save_path
        })


In [ ]:
from gru_model import GRULanguageModel
from lstm_model import LSTMLanguageModel
from rnn_model import RNNLanguageModel
from transformer_model import TransformerLanguageModel

run_experiments("gru", GRULanguageModel, hyperparams_grid["gru"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("lstm", LSTMLanguageModel, hyperparams_grid["lstm"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("rnn", RNNLanguageModel, hyperparams_grid["rnn"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("transformer", TransformerLanguageModel, hyperparams_grid["transformer"], tokenizer, train_loader, test_loader, DEVICE)



 Training gru_0_1744794807 with config: {'embed_dim': 128, 'hidden_dim': 256, 'num_layers': 2, 'lr': 0.001}


/home/vjosep3/.local/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/50: 100%|██████████| 155/155 [00:08<00:00, 18.03it/s]


Epoch 1 | Train Loss: 6.9873 | Val Loss: 6.6835
 Model saved to gru_0_1744794807.pt


Epoch 2/50: 100%|██████████| 155/155 [00:08<00:00, 19.14it/s]


Epoch 2 | Train Loss: 6.6077 | Val Loss: 6.4148
 Model saved to gru_0_1744794807.pt


...

Epoch 34/50: 100%|██████████| 155/155 [00:14<00:00, 10.75it/s]


Epoch 34 | Train Loss: 4.3838 | Val Loss: 4.5135
 Model saved to gru_1_1744795270.pt


Epoch 35/50: 100%|██████████| 155/155 [00:14<00:00, 10.90it/s]


Epoch 35 | Train Loss: 4.3671 | Val Loss: 4.5101
 Model saved to gru_1_1744795270.pt


Epoch 36/50:  79%|███████▊  | 122/155 [00:11<00:03, 10.88it/s]

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("experiment_results.csv", index=False)
results_df.sort_values(by="perplexity").head()
